In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import scipy as sc
import statsmodels.api as sm
import scipy.stats as stats
import cartopy.crs as ccrs
import pygrib
import cfgrib
import stormeunice as eun

sns.set_theme(style="white")
sns.set_style("white")

random.seed(10)

In [2]:
dir(eun.data.Data)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'create_latlon_grid',
 'get_friday_data',
 'get_latlon',
 'load_meta']

# Import data

In [3]:
directory, experiments, inits, cfpf = eun.data.Data.load_meta()
lat, lon = eun.data.Data.get_latlon()
south_df = eun.data.Data.get_friday_data()

In [ ]:
south_df.head()